In [1]:
import pandas as pd
import os
import boto3
import spacy
import pandas as pd
import re
import polars as pl
import unicodedata
from dotenv import load_dotenv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spedrickcase\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spedrickcase\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\spedrickcase\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
RUN_AWS_FUNCTIONS_S3 = os.getenv("RUN_AWS_FUNCTIONS_S3", "0")

In [3]:
if RUN_AWS_FUNCTIONS_S3 == "1":

    try:
        s3_client = boto3.client("s3", region_name="eu-west-2")
        print("s3_client is initialized:", s3_client)
    except Exception as e:
        print(f"Error initializing s3_client: {e}")
        raise e

    TMP_DIR = "/tmp/"

    def download_file_from_s3(bucket_name, key, download_path):
        """Download a file from S3 to the local filesystem."""
        s3_client.download_file(bucket_name, key, download_path)
        print(f"Downloaded {key} to {download_path}")

    def upload_file_to_s3(file_path, bucket_name, key):
        """Upload a file to S3."""
        s3_client.upload_file(file_path, bucket_name, key)
        print(f"Uploaded {file_path} to {key}")

In [4]:
if RUN_AWS_FUNCTIONS_S3 == "0":
    latest_run_path = 'latest_run.env'
else:
    latest_run_path = 'latest_run_s3.env'

load_dotenv(latest_run_path)

# Load the .env file
env_file_path = os.getenv("env_file_path") # Update this to your actual path
print(env_file_path)
load_dotenv(env_file_path)

# Access the variables
save_file_suffix = os.getenv('save_file_suffix')
time_queries_folder = os.getenv('time_queries_folder')
topics_outputs_suffix = os.getenv('topics_outputs_suffix')
task = os.getenv('task')
panel_time_begin = os.getenv('panel_time_begin')
panel_time_end = os.getenv('panel_time_end')
one_year_before_time_begin = os.getenv('one_year_before_time_begin')
two_years_before_time_begin = os.getenv('two_years_before_time_begin')
three_years_before_time_begin = os.getenv('three_years_before_time_begin')
six_months_before_time_begin = os.getenv('six_months_before_time_begin')
six_months_before_time_end = os.getenv('six_months_before_time_end')
six_months_after_time_end = os.getenv('six_months_after_time_end')
time_begin = os.getenv('time_begin')
time_end = os.getenv('time_end')
case_note_time_begin = os.getenv('case_note_time_begin')

asc_bucket = os.getenv('asc_bucket')
asc_folder = os.getenv('asc_folder')
training_run_folder = os.getenv('training_run_folder')
training_run_tfidf_folder = os.getenv('training_run_tfidf_folder')
training_run_modelling_folder = os.getenv('training_run_modelling_folder')
training_run_embedding_folder = os.getenv('training_run_embedding_folder')
training_run_model_file = os.getenv('training_run_model_file')

training_time_period = os.getenv('training_time_period')

run_name = os.getenv('run_name')
run_base_folder = os.getenv('run_base_folder')
relative_file_prefix = os.getenv('relative_file_prefix')
queries_sql_folder = os.getenv('queries_sql_folder')
relative_queries_sql_folder = os.getenv('relative_queries_sql_folder')

case_note_app_load_folder = os.getenv('case_note_app_load_folder')
conda_environment_case_note_load = os.getenv('conda_environment_case_note_load')
bat_file_location = os.getenv('bat_file_location')

topic_model_app_folder = os.getenv('topic_model_app_folder')
custom_regex_file_location = os.getenv('custom_regex_file_location')
split_sentences_suffix = os.getenv('split_sentences_suffix')

output_data_folder = os.getenv('output_data_folder')
relative_data_folder = os.getenv('relative_data_folder')
output_data_with_query_date_folder = os.getenv('output_data_with_query_date_folder')
output_processed_data_folder = os.getenv('output_processed_data_folder')
topics_data_folder = os.getenv('topics_data_folder')
tfidf_data_folder = os.getenv('tfidf_data_folder')
embeddings_folder = os.getenv('embeddings_folder')
model_folder = os.getenv('model_folder')

# Example usage
print(save_file_suffix)
print(time_queries_folder)
print(topics_outputs_suffix)

c:\Users\spedrickcase\OneDrive - Lambeth Council\2024\ASC Project\asc_predict\run_2025-04-01_2025-04-07_inference.env
_25_04_07
25_04_07
20250407


In [5]:
if not os.path.exists(output_processed_data_folder): os.makedirs(output_processed_data_folder)
if not os.path.exists(topics_data_folder): os.makedirs(topics_data_folder)
if not os.path.exists(tfidf_data_folder): os.makedirs(tfidf_data_folder)
if not os.path.exists(embeddings_folder): os.makedirs(embeddings_folder)
if not os.path.exists(model_folder): os.makedirs(model_folder)

output_file_path_df = pd.read_csv(output_data_with_query_date_folder + "/output_file_list_df.csv")

def replace_up_to_last_slash(string, replacement):
    # Use a regex pattern that matches both '/' and '\'
    string = string.encode('unicode_escape').decode('ascii')
    string_split = string.split("/")[1]
    out_path = replacement + "/" + string_split
    return out_path

result = output_file_path_df['table_path'].apply(lambda x: replace_up_to_last_slash(x, output_data_with_query_date_folder))

output_file_path_df['table_path'] = result

output_file_path_df = output_file_path_df.set_index('table_name')['table_path'].to_dict() # Converted to dictionary

unique_person_id_list_df = pd.read_csv(output_data_with_query_date_folder + "/unique_person_id_list.csv")
unique_subgroup_id_list_df = pd.read_csv(output_data_with_query_date_folder + "/unique_subgroup_id_list.csv")
unique_form_id_list_df = pd.read_csv(output_data_with_query_date_folder + "/unique_form_id_list.csv")

In [6]:
function_output_folder = tfidf_data_folder

##### https://journals.lww.com/ccejournal/fulltext/2021/06000/impact_of_different_approaches_to_preparing_notes.7.aspx

In [7]:
stop_words = set(stopwords.words('english'))
# List of words to remove from the stopword set
words_to_remove = ['no', 'nor', 'not', 'don', 'don’t', 'wasn', 'wasn’t', 'weren', 'weren’t', "don't", "wasn't", "weren't"]

# Remove the specified words from the stopwords set
for word in words_to_remove:
    stop_words.discard(word.lower())
    
stemmer = PorterStemmer()

In [8]:


# Load the small English model (which includes tokenization and lemmatization)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "tok2vec", "tagger", "attribute_ruler"])

# def clean_text_spacy(text):
#     """Cleans text by removing HTML tags, extra spaces, and non-alphabetic characters."""
#     # Remove HTML tags and special characters
#     text = re.sub(r'<.*?>|&nbsp;|\r\n|&lt;|&gt;|<strong>|</strong>', ' ', text)
#     text = text.replace(u'\xa0', ' ')  # Replace non-breaking space
#     return ' '.join(text.split())  # Normalize spaces

regex_pattern_1 = r'<.*?>|&nbsp;|\r\n|&lt;|&gt;|<strong>|</strong>'
regex_pattern_2 = u'\xa0'


def initial_clean(texts, custom_regex=''):

    for text in texts:
        if not text or pd.isnull(text):
            text = ""

        # Normalize unicode characters to decompose any special forms
        normalized_text = unicodedata.normalize('NFKC', text)

        # Replace smart quotes and special punctuation with standard ASCII equivalents
        replacements = {
            '‘': "'", '’': "'", '“': '"', '”': '"', 
            '–': '-', '—': '-', '…': '...', '•': '*',
        }

        # Perform replacements
        for old_char, new_char in replacements.items():
            normalised_text = normalized_text.replace(old_char, new_char)

        text = normalised_text

    # Convert to polars Series
    texts = pl.Series(texts).str.strip_chars()
    
    # Define a list of patterns and their replacements
    patterns = [
        (regex_pattern_1, " "),
        (regex_pattern_2, " ")
    ]
    
    # Apply each regex replacement
    for pattern, replacement in patterns:
        texts = texts.str.replace_all(pattern, replacement)
    
    # Convert the series back to a list
    texts = texts.to_list()
    
    return texts

def process_texts_spacy(texts):
    """Applies cleaning and lemmatization to a batch of texts."""
    cleaned_texts = initial_clean(texts)

    #print("pre clean complete")
    #print(nlp.pipeline)
    
    # Get stopwords set once, outside the loop
    stop_words = nlp.Defaults.stop_words
    
    docs = nlp.pipe(cleaned_texts, batch_size=100, n_process=4)
    
    #print("Text has been lemmatised")
    
    processed_texts = []
    for doc in docs:
        # Using spaCy's vocab for faster lookups
        valid_tokens = [token.lemma_ for token in doc if token.is_alpha and token.text not in stop_words]
        processed_texts.append(' '.join(valid_tokens))
    
    return processed_texts

def process_data(df, col):
    """Processes a DataFrame column using spaCy for faster text processing."""
    df['stemmed_' + col] = process_texts_spacy(df[col].tolist())  # Convert to list for batch processing
    return df[['PERSON_ID', 'DECISION_GROUP', 'stemmed_' + col]]


def merge_note_form(case_note_df, joined_table, start='GROUP_DECISION_DATE', end="GROUP_DECISION_DATE_PLUS_6", out_col_name="case_note"):
    case_note_df['PERSON_ID'] = case_note_df['PERSON_ID'].astype(str)
    case_note_df['case_note'] = case_note_df['case_note'].fillna("")

    merged_df = pd.merge(case_note_df, joined_table, on=['PERSON_ID'], how='inner')

    mask = (merged_df['created_on'] >= merged_df[start]) & \
                    (merged_df['created_on'] < merged_df[end]) & \
                    (merged_df['created_on'] < merged_df['NEXT_DECISION_DATE'])
    df_case_notes = (merged_df[mask]
                              .groupby(['PERSON_ID', 'DECISION_GROUP', start, end]).agg({
                                        "case_note": ' '.join}) #'Time period' if we want to split times
                              .reset_index())#names=out_col_name))
    print(df_case_notes.head())
    df_case_notes.rename(columns={"case_note":out_col_name}, inplace=True)
    df_case_notes.drop(columns=[start,end], axis=1, inplace=True, errors="ignore")
    final_df = pd.merge(df_case_notes, joined_table, on=['PERSON_ID','DECISION_GROUP'], how='inner')
    #finaldf= finaldf.fillna("")
    return final_df

def tfidf(df, column, topn, vocab_save_file='', predefined_vocab_file=''):
    # If you have a predefined vocabulary, load from file.
    if predefined_vocab_file:
        top_terms = pd.read_csv(predefined_vocab_file)
    else:
        vectorizer = CountVectorizer(stop_words='english')

        # Fit and transform the text data
        X = vectorizer.fit_transform(df[column])

        # Get feature names (terms)
        terms = vectorizer.get_feature_names_out()

        # Sum the term frequencies across all documents
        term_frequencies = X.toarray().sum(axis=0)
        term_freq_df = pd.DataFrame({'term': terms, 'frequency': term_frequencies})
        top_terms = term_freq_df.sort_values(by='frequency', ascending=False).head(topn)

        # Save vocabulary as a csv file
        if vocab_save_file:
            top_terms.to_csv(vocab_save_file)
    
    # Extract the top n terms into a list
    top_terms_list = top_terms['term'].tolist()
    tfidf_vectorizer = TfidfVectorizer(vocabulary=top_terms_list)

    # Check if the column is empty or contains only null values
    if df[column].isnull().all() or df[column].eq('').all():
        # Create a zero matrix with the same number of features as top_terms_list
        X_tfidf = csr_matrix((len(df), len(top_terms_list)))  # Create a sparse matrix
    else:
        #tfidf_vectorizer = TfidfVectorizer(vocabulary=top_terms_list)
        # Fit and transform the text data using TF-IDF
        X_tfidf = tfidf_vectorizer.fit_transform(df[column])

    # Convert TF-IDF matrix to a DataFrame
    tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
    return tfidf_df

def get_tfidf(df, column, topn=1000, vocab_save_file='tfidf_vocabulary', predefined_vocab_file=''):
    '''Wrapper function for tfidf'''
    tfidf_df = tfidf(df, column, topn, vocab_save_file, predefined_vocab_file)

    # Add the TF-IDF features back to the original DataFrame
    df_with_tfidf = pd.concat([df, tfidf_df], axis=1)
    df_with_tfidf = df_with_tfidf.drop([column], errors="ignore", axis=1)
    return df_with_tfidf

def unique_token_count(text):
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    unique_tokens = set(tokens)           # Find unique tokens
    return len(unique_tokens)

In [9]:
joined_table = pd.read_parquet(output_processed_data_folder + "/joined_table_review_filtered" + save_file_suffix + "_six_month_no_added_features.parquet")
joined_table['PERSON_ID'] = joined_table['PERSON_ID'].astype(str)

for idx, val in enumerate(joined_table.loc[:,'CHANGE_IN_NEXT_GROUP'].value_counts()):
    print(f"Class={idx}, Count={val}, Percentage={((val/ joined_table.shape[0]) * 100):.1f}%")

In [10]:
case_note_first_three_df = pd.read_csv(tfidf_data_folder + '/filtered_case_notes_first_three_months_f' + save_file_suffix +  '_clean_anon_' + topics_outputs_suffix + '.csv')
case_note_three_six_df = pd.read_csv(tfidf_data_folder + '/filtered_case_notes_three_to_six_months_f' + save_file_suffix +  '_clean_anon_' + topics_outputs_suffix + '.csv')
case_note_df = pd.concat([case_note_first_three_df,case_note_three_six_df], ignore_index=True).iloc[:,2:]
case_note_df['PERSON_ID'] = case_note_df['PERSON_ID'].astype(str)

#how_are_you = pd.read_csv(tfidf_data_folder+ '/joined_table_out_panel_how_are_you' + save_file_suffix +  '_clean_anon_' + topics_outputs_suffix + '.csv').iloc[:,2:]
#how_are_you['PERSON_ID'] = how_are_you['PERSON_ID'].astype(str)
#how_are_you= how_are_you.fillna('')

In [11]:
#how_are_you.columns

In [12]:
case_note_df[case_note_df['PERSON_ID'] == '2305']

,PERSON_ID,created_on,note_type,title,case_note


In [13]:
joined_table['REQUESTS']

0       Update - Home care mobilization telephone disc...
1       Requesting SH OD POC with personal care, meals...
2       Fikret's parents, who are advancing in age and...
3       CARE AND SUPPORT PLACEMENT REVIEW\n\nOUTCOMES ...
4       What are you requesting?\n*Long Term x 4 calls...
                              ...                        
2259    OUTCOMES AND RECOMMENDATIONS:\n\n1.For Mr. Sim...
2260    Recommendation.\n\nCommunity Duty received a r...
2261    Reduction in care Monday-Friday as family will...
2262                                 No change in service
2263    Request to reduce 1:1 hours from 19hrs to 14hr...
Name: REQUESTS, Length: 2264, dtype: object

In [14]:
case_note_df

,PERSON_ID,created_on,note_type,title,case_note
0,2375,2025-03-17 15:50:30,PHONE,Unexplained discomfort personal care,Reported to me by staff members that ET has sp...
1,2375,2025-01-15 20:40:24,PHONE,Eugenie home late from the Day service,informed that due to transport bus breaking ...
2,2375,2025-02-04 11:13:25,PHONE,Homecare Mobilisation: Telephone call received,Received a call from (carer) regarding transf...
3,2375,2025-02-17 12:36:50,PHONE,Updated Light touch telephone review conversat...,Updated Light touch telephone review conversat...
4,2375,2025-03-05 14:20:27,PHONE,T/C update with the sister about Eugenie meeti...,"I liaised with her sister, who i informed tha..."
...,...,...,...,...,...
7637,1530517,2024-11-15 15:29:34,PHONE,CDT - T/C received from daughter RE: Issues wi...,CDT - T/C received from daughter reporting iss...
7638,1533432,2024-10-29 10:23:07,CSEVENT,email regarding booking review,", Following our conversation , I will be send..."
7639,1533432,2024-10-29 10:19:42,CSEVENT,review documents sent,", See attached s review documents and suppor..."
7640,1533432,2024-10-29 10:21:39,CSEVENT,review meeting booked,"All, The review meeting for has been booked _"


In [15]:
if training_time_period == "first_six_months":
    print("TFIDF for six months post previous review")
    merged_df_0_6 = merge_note_form(case_note_df, joined_table, 'GROUP_DECISION_DATE', 'GROUP_DECISION_DATE_PLUS_6', 'case_notes_0_6')
    merged_df_0_6

    merged_df_0_3 = merge_note_form(case_note_df, joined_table, 'GROUP_DECISION_DATE', 'GROUP_DECISION_DATE_PLUS_3', 'case_notes_0_3')
    merged_df_0_3

    merged_df_3_6 = merge_note_form(case_note_df, joined_table, 'GROUP_DECISION_DATE_PLUS_3', 'GROUP_DECISION_DATE_PLUS_6', 'case_notes_3_6')
    merged_df_3_6

elif training_time_period == "latest_six_months":
    print("TFIDF for six months before next review")
    merged_df_0_6 = merge_note_form(case_note_df, joined_table, 'PREDICTION_DATE_LESS_6', 'PREDICTION_DATE', 'case_notes_0_6')
    merged_df_0_6

    merged_df_0_3 = merge_note_form(case_note_df, joined_table, 'PREDICTION_DATE_LESS_3', 'PREDICTION_DATE', 'case_notes_0_3')
    merged_df_0_3

    merged_df_3_6 = merge_note_form(case_note_df, joined_table, 'PREDICTION_DATE_LESS_6', 'PREDICTION_DATE_LESS_3', 'case_notes_3_6')
    merged_df_3_6

TFIDF for six months before next review
  PERSON_ID  DECISION_GROUP PREDICTION_DATE_LESS_6 PREDICTION_DATE  \
0   1291796               4    2024-10-28 16:28:00      2025-04-07   
1   1392476               3    2024-12-05 08:51:00      2025-04-07   
2   1394107               3    2024-10-09 00:00:00      2025-04-07   
3    146559               4    2025-02-14 16:04:00      2025-04-07   
4    205255               2    2024-10-09 00:00:00      2025-04-07   

                                           case_note  
0   , I hope you are well. The above-named client...  
1  Conversation with  about having a spare key fo...  
2   sent to  regarding his mother .  Transitionin...  
3  Assessor went to  ward on  / / to speak to . O...  
4   asked for  email as  (aunt) has received docu...  
  PERSON_ID  DECISION_GROUP PREDICTION_DATE_LESS_3 PREDICTION_DATE  \
0   1291796               4    2025-01-07 00:00:00      2025-04-07   
1   1392476               3    2025-01-07 00:00:00      2025-04-07   

In [16]:
#merged_df = merged_df_0_6.merge(how_are_you, on=['PERSON_ID', 'DECISION_GROUP'], how='left')
#merged_df = merged_df.merge(merged_df_0_3[['PERSON_ID', 'DECISION_GROUP', 'case_notes_0_3']], on=['PERSON_ID', 'DECISION_GROUP'], how='left')
#merged_df = merged_df.merge(merged_df_3_6[['PERSON_ID', 'DECISION_GROUP', 'case_notes_0_3']], on=['PERSON_ID', 'DECISION_GROUP'], how='left')

In [17]:
stemmed_0_6 = process_data(merged_df_0_6, 'case_notes_0_6')

In [18]:
stemmed_0_3 = process_data(merged_df_0_3, 'case_notes_0_3')

In [19]:
stemmed_3_6 = process_data(merged_df_3_6, 'case_notes_3_6')

In [20]:
#stemmed_how_are_you = process_data(how_are_you, 'GROUP_HOW_ARE_YOU')

In [21]:
joined_table['REQUESTS']= joined_table['REQUESTS'].fillna('')
stemmed_requests = process_data(joined_table, 'REQUESTS')

In [22]:
#stemmed_how_are_you

In [23]:
stemmed_requests

,PERSON_ID,DECISION_GROUP,stemmed_REQUESTS
0,10005,2,update home care mobilization telephone discus...
1,1008583,1,requesting sh od poc personal care meals paper...
2,101096,2,fikret parents advancing age experiencing heal...
3,10193,2,care and support placement review outcomes and...
4,102240,3,what requesting long term x calls daily dh poc...
...,...,...,...
2259,95003,1,outcomes and recommendations simms reside curr...
2260,95204,2,recommendation community duty received referra...
2261,97693,2,reduction care monday friday family support we...
2262,98394,3,no change service


In [24]:
if (task == "eval") | (task == "inference"):
    final_case_note_df = get_tfidf(stemmed_0_6, 'stemmed_case_notes_0_6', 1000, vocab_save_file='', predefined_vocab_file=training_run_tfidf_folder + '/case_note_0_6_tfidf_vocabulary.csv')
    final_case_note_0_3_df = get_tfidf(stemmed_0_3, 'stemmed_case_notes_0_3', 1000, vocab_save_file='', predefined_vocab_file=training_run_tfidf_folder + '/case_note_0_3_tfidf_vocabulary.csv')
    final_case_note_3_6_df = get_tfidf(stemmed_3_6, 'stemmed_case_notes_3_6', 1000, vocab_save_file='', predefined_vocab_file=training_run_tfidf_folder + '/case_note_3_6_tfidf_vocabulary.csv')

    #final_how_are_you_df = get_tfidf(stemmed_how_are_you, 'stemmed_GROUP_HOW_ARE_YOU', 1000, vocab_save_file='', predefined_vocab_file=training_run_tfidf_folder + '/how_are_you_tfidf_vocabulary.csv')
    final_requests_df = get_tfidf(stemmed_requests, 'stemmed_REQUESTS', 500, vocab_save_file='', predefined_vocab_file=training_run_tfidf_folder + '/requests_tfidf_vocabulary.csv')

elif task == "training":
    final_case_note_df = get_tfidf(stemmed_0_6, 'stemmed_case_notes_0_6', 1000, vocab_save_file=tfidf_data_folder + '/case_note_0_6_tfidf_vocabulary.csv', predefined_vocab_file='')
    final_case_note_0_3_df = get_tfidf(stemmed_0_3, 'stemmed_case_notes_0_3', 1000, vocab_save_file=tfidf_data_folder + '/case_note_0_3_tfidf_vocabulary.csv', predefined_vocab_file='')
    final_case_note_3_6_df = get_tfidf(stemmed_3_6, 'stemmed_case_notes_3_6', 1000, vocab_save_file=tfidf_data_folder + '/case_note_3_6_tfidf_vocabulary.csv', predefined_vocab_file='')
    
    #final_how_are_you_df = get_tfidf(stemmed_how_are_you, 'stemmed_GROUP_HOW_ARE_YOU', 1000, vocab_save_file=tfidf_data_folder + '/how_are_you_tfidf_vocabulary.csv', predefined_vocab_file='')
    final_requests_df = get_tfidf(stemmed_requests, 'stemmed_REQUESTS', 500, vocab_save_file=tfidf_data_folder + '/requests_tfidf_vocabulary.csv', predefined_vocab_file='')

In [25]:
#final_case_note_df = final_case_note_df.drop(['Stemmed_HowareYou', 'Stemmed_requests'], errors="ignore", axis=1)
#final_how_are_you_df = final_how_are_you_df.drop(['Stemmed_CaseNote','Stemmed_requests'], errors="ignore", axis=1)
#final_requests_df = final_requests_df.drop(['Stemmed_CaseNote', 'Stemmed_HowareYou'], errors="ignore", axis=1)

In [26]:
#final_case_note_df.head(2)

In [27]:
final_case_note_df.to_parquet(tfidf_data_folder + "/tfidf_case_note_6mo_1000" + save_file_suffix + ".parquet", index=None)
final_case_note_0_3_df.to_parquet(tfidf_data_folder + "/tfidf_case_note_0_3mo_1000" + save_file_suffix + ".parquet", index=None)
final_case_note_3_6_df.to_parquet(tfidf_data_folder + "/tfidf_case_note_3_6mo_1000" + save_file_suffix + ".parquet", index=None)
#final_how_are_you_df.to_parquet(tfidf_data_folder + "/tfidf_how_are_you_1000" + save_file_suffix + ".parquet", index=None)
final_requests_df.to_parquet(tfidf_data_folder + "/tfidf_requests_500" + save_file_suffix + ".parquet", index=None)

In [28]:
if RUN_AWS_FUNCTIONS_S3 == "1":
    print("Now uploading files from:", function_output_folder)

    # Upload output files back to S3
    for root, _, files in os.walk(function_output_folder):
        for file_name in files:
            print("file_name:", file_name)
            local_file_path = os.path.join(root, file_name)
            output_key = f"{tfidf_data_folder}/{file_name}"
            print("Output location is:", output_key)
            upload_file_to_s3(local_file_path, asc_bucket, output_key)
else:
    print("Moving files from output folder to relevant folder location")

    print("function_output_folder:", function_output_folder)

    # Not necessary to do this locally
    # # Move files to relevant folder - 
    # for root, _, files in os.walk(function_output_folder + "/"):
    #     for file_name in files:
    #         print("file_name:", file_name)
    #         local_file_path = os.path.join(root, file_name)
    #         destination_file = f"{function_output_folder}/{file_name}"
    #         print("Output location is:", destination_file)
    #         shutil.move(local_file_path, destination_file)

Moving files from output folder to relevant folder location
function_output_folder: c:\Users\spedrickcase\OneDrive - Lambeth Council\2024\ASC Project\asc_predict\run\run_2025-04-01_2025-04-07_inference\processed_data\tfidf
